In [659]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [660]:
df = pd.read_csv('new_expenditure_data.csv')

In [661]:
q1 = df.loc[(df['Year'] == 2020) & (df['Quarter'] == 1)] 
q1 = q1[['Constituency name', '3-Travel', "1-Employees' salaries", "4-Hospitality", "2-Service Contracts",'Name']]
q1.rename(columns={
    'Constituency name': 'Constituency',
    '3-Travel': 'Travel',
    "1-Employees' salaries": "Salaries",
    "4-Hospitality": "Hospitality",
    "2-Service Contracts": "Contracts",
    "party": "Caucus"
}, inplace=True)

In [662]:
q1['Year'] = 2020
q1['Quarter'] = 1
q2 = pd.read_csv('MembersExpenditures.2021Q2.csv')
q2['Year'] = 2020
q2['Quarter'] = 2
q3 = pd.read_csv('MembersExpenditures.2021Q3.csv')
q3['Year'] = 2020
q3['Quarter'] = 3
q4 = pd.read_csv('MembersExpenditures.2021Q4.csv')
q4['Year'] = 2020
q4['Quarter'] = 4

new_df = pd.concat([q1, q2, q3, q4])
new_df.drop(columns=['Caucus'], inplace=True)

In [663]:
new_df
new_df['Name'] = new_df['Name'].replace(r'\s+', '', regex=True)


In [664]:
new_df.loc[(new_df['Quarter'] <= 4)].groupby(['Name']).sum()
new_df.loc[new_df['Name'].str.contains('Elizabeth')]


,Constituency,Travel,Salaries,Hospitality,Contracts,Name,Year,Quarter
205,Saanich—Gulf Islands,13792.27,72004.65,0.0,568.74,"May,Elizabeth",2020,1
207,Saanich—Gulf Islands,5031.92,93288.50,0.0,11739.41,"May,Elizabeth",2020,2
207,Saanich—Gulf Islands,17479.51,52370.36,0.0,13860.15,"May,Elizabeth",2020,3
209,Saanich—Gulf Islands,13555.71,96296.82,0.0,43687.74,"May,Elizabeth",2020,4


In [665]:
# q2
a = new_df.loc[(new_df['Quarter'] <= 2)].groupby(['Name', 'Constituency']).sum().reset_index()
a['Year'] = 2020
a['Quarter'] = 2
b = new_df.loc[(new_df['Quarter'] <= 3)].groupby(['Name', 'Constituency']).sum().reset_index()
b['Year'] = 2020
b['Quarter'] = 3
c = new_df.loc[(new_df['Quarter'] <= 4)].groupby(['Name', 'Constituency']).sum().reset_index()
c['Year'] = 2020
c['Quarter'] = 4

In [666]:
new_df = pd.concat([new_df.loc[new_df['Quarter'] == 1], a, b, c])

In [667]:
new_df.rename(columns={
    'Constituency': 'Constituency name',
    'Travel': '3-Travel',
    "Salaries": "1-Employees' salaries",
    "Hospitality": "4-Hospitality",
    "Contracts": "2-Service Contracts",
    "Caucus": "party"
}, inplace=True)

In [668]:
new_df

,Constituency name,3-Travel,1-Employees' salaries,4-Hospitality,2-Service Contracts,Name,Year,Quarter
0,Edmonton Manning,9473.52,53194.10,0.00,500.00,"Aboultaif,Ziad",2020,1
1,Parry Sound—Muskoka,1673.62,51822.64,0.00,0.00,"Aitchison,Scott",2020,1
2,Central Okanagan—Similkameen—Nicola,8252.41,66389.21,127.86,1264.30,"Albas,Dan",2020,1
3,Mississauga Centre,4960.12,50896.13,0.00,284.15,"Alghabra,Hon.Omar",2020,1
4,Aurora—Oak Ridges—Richmond Hill,8904.82,34508.90,0.00,942.28,"Alleslev,Leona",2020,1
...,...,...,...,...,...,...,...,...
356,Cumberland—Colchester,2709.97,285643.20,0.00,85154.05,"Zann,Lenore",2020,4
357,Prince George—Peace River—Northern Rockies,96500.40,321359.02,1384.92,108168.87,"Zimmer,Bob",2020,4
358,Pierrefonds—Dollard,9501.76,270479.17,2653.26,81416.43,"Zuberi,Sameer",2020,4
359,West Nova,30413.11,242665.97,144.85,79617.86,"d'Entremont,Chris",2020,4


In [669]:
df.drop(df[df['Year'] == 2020].index, inplace=True)
dx = df.append(new_df, sort=True)

In [670]:
dx['Name'] = dx['Name'].replace(r'\s+', '', regex=True)
dx = dx.set_index(['Constituency name', 'Name', 'Year', 'Quarter']).sort_index().reset_index()

In [671]:
dx.loc[dx['Number of electors'] == 0.0, ['Number of electors']] = np.NaN
dx = dx.fillna(method='ffill', limit=5)
df = dx

In [672]:
df['Name'] = df['Name'].apply(lambda x: x.split('\t\t')[0].replace('Hon. ', ''))
df['minister'] = df['Name'].str.match(r'.*,Hon\..*')
df['Name'] = df['Name'].str.replace(r'Hon\.\s', '', regex=True)
df['Name'] = df['Name'].str.replace(r'Hon\.', '', regex=False)
df['Name'] = df['Name'].str.replace(r'Right Hon\.\s', '', regex=True)


In [673]:
df.loc[df['Name'].str.contains('Fast')][['Name', 'Year', 'Quarter', 'Number of electors']]


,Name,Year,Quarter,Number of electors
0,"Fast,Hon.Ed",2012,4,85143.0
1,"Fast,Hon.Ed",2013,4,85143.0
2,"Fast,Hon.Ed",2014,1,82332.0
3,"Fast,Hon.Ed",2014,2,85143.0
4,"Fast,Hon.Ed",2014,3,85143.0
5,"Fast,Hon.Ed",2014,4,85143.0
6,"Fast,Hon.Ed",2015,1,85143.0
7,"Fast,Hon.Ed",2015,2,85143.0
8,"Fast,Hon.Ed",2015,3,69299.0
9,"Fast,Hon.Ed",2015,4,69299.0


In [674]:
df['Quarter'] = df['Quarter'].apply(lambda x: np.NaN if x == '4' else int(x))
df.drop(df.loc[df['Quarter'] == 'Quarter'].index, inplace=True)
df.drop(columns=['Status'], inplace=True)

In [675]:
df['Member\'s per diem expenses'].fillna(0.0, inplace=True)

In [676]:
df['Member'] = df['Member'].astype(float)
df['Designated traveller'] = df['Designated traveller'].astype(float)
df['Dependants'] = df['Dependants'].astype(float)
df['Employees'] = df['Employees'].astype(float)
df['Member\'s accommodation expenses'] = df['Member\'s accommodation expenses'].astype(float)
df['Member\'s per diem expenses'] = df['Member\'s per diem expenses'].astype(float)
df['Member\'s secondary residence expenses']= df['Member\'s secondary residence expenses'].astype(float)
# df.fillna(0.0, inplace=True)
# df['3-Travel'] = df['Member'] + df['Designated traveller'] + df['Dependants'] + df['Employees'] + df['Member\'s accommodation expenses'] + df['Member\'s per diem expenses'] + df['Member\'s secondary residence expenses']

In [677]:
df.groupby(['Name', 'Year', 'Quarter'])['Total'].sum().sort_values(ascending=False).head(10)

Name                 Year  Quarter
Doherty,Todd         2018  4          622466.23
Fletcher,Hon.Steven  2012  4          612259.38
Nault,Hon.Robert     2017  4          606651.93
Yurdiga,David        2018  4          600262.77
Gill,Marilène        2020  3          596939.07
                           2          596939.07
                           1          596939.07
                     2019  4          596939.07
                     2020  4          596939.07
Nault,Hon.Robert     2018  4          591401.04
Name: Total, dtype: float64

In [678]:
df.rename(columns={
    'Year': 'year',
    'Quarter': 'quarter',
    'Name': 'name',
    'Constituency name': 'constituency_name',
    'Constituency size': 'constituency_size',
    'Number of electors': 'number_of_electors', 
    "1-Employees' salaries": '_employees_salaries',
    '2-Service Contracts': '_service_contracts',
    '3-Travel': '_travel',
    'Member': 'member',
    'Designated traveller': 'designated_traveller',
    'Dependants': 'dependants',
    'Employees': 'employees',
    "Member's accommodation expenses":  'member_accommodation_expenses',
    "Member's per diem expenses": 'member_per_diem_expenses',
    "Member's secondary residence expenses": 'member_secondary_residence_expenses',
    '4-Hospitality': '_hospitality',
    '5-Gifts': '_gifts',
    '6-Advertising': '_advertising',
    '7-Printing': '_printing',
    'Householders': 'householders',
    'Ten percenters': 'ten_percenters',
    'Other printing related expenses': 'other_printing_related_expenses',
    '8-Offices': '_offices',
    'Constituency office leases, insurance and utilities': 'constituency_office_leases_insurance_and_utilities',
    'Furniture, furnishing and equipment purchases': 'furniture_furnishing_and_equipment_purchases',
    'Equipment rentals': 'equipment_rentals',
    'Informatics and telecommunication equipment purchases': 'informatics_and_telecommunication_equipment_purchases',
    'Telecommunication services':  'telecommunication_services',
    'Repairs and maintenance': 'repairs_and_maintenance',
    'Postage and courier services': 'postage_and_courier_services',
    'Materials and supplies': 'materials_and_supplies',
    'Training': 'training',
    'Total': 'total'
}, inplace=True)


In [679]:
ministers = pd.read_pickle('ministers.pkl')
distances = pd.read_pickle('district_distances.pkl')

In [680]:
df['name'] = df['name'].replace(r'\s+', '', regex=True)

In [681]:
df.loc[(df['year'] == 2019) & (df['quarter'] == 4)].sample(5)

,constituency_name,name,year,quarter,_employees_salaries,_service_contracts,_travel,_hospitality,_gifts,_advertising,...,member_secondary_residence_expenses,number_of_electors,other_printing_related_expenses,postage_and_courier_services,repairs_and_maintenance,telecommunication_services,ten_percenters,total,training,minister
8630,Saskatoon—University,"Trost,Brad",2019,4,166495.08,1171.79,83874.23,2269.49,233.50,1334.00,...,11593.90,58394.0,0.00,273.74,143.10,4521.00,960.98,290144.53,0.00,False
9934,Trois-Rivières,"Aubin,Robert",2019,4,134883.79,1723.50,15503.73,4513.24,154.95,12597.54,...,6089.16,91098.0,449.00,530.45,3228.02,4271.52,3325.55,206317.36,0.00,False
8151,Rosemont—La Petite-Patrie,"Boulerice,Alexandre",2019,4,288738.72,40540.57,37765.66,1025.76,0.00,6555.02,...,22450.00,85832.0,1078.75,671.27,0.00,6454.40,3919.61,443689.26,5141.00,False
5812,Mississauga East—Cooksville,"Fonseca,Peter",2019,4,237792.96,44191.23,39081.04,6469.22,0.00,22349.45,...,15516.77,86191.0,6517.34,3067.23,21430.15,9771.51,12166.32,457885.89,0.00,False
901,Bow River,"Shields,Martin",2019,4,244889.45,2395.38,68315.82,1637.08,779.40,24423.23,...,15410.99,80610.0,1080.00,3067.28,724.50,11955.24,3631.72,412315.29,1458.69,False


In [682]:
df = df.set_index('name').join(ministers[['name', 'parliament', 'party']].set_index('name'), how='left').reset_index()
df = df.set_index('constituency_name').join(distances.set_index('constituency_name')).reset_index()
df.drop_duplicates(subset=['name', 'year', 'quarter'], inplace=True)
df.dropna(subset=['number_of_electors'], inplace=True)



In [683]:
a

,Name,Constituency,Travel,Salaries,Hospitality,Contracts,Year,Quarter
0,"Aboultaif,Ziad",Edmonton Manning,25951.88,114017.11,394.59,19436.14,2020,2
1,"Aitchison,Scott",Parry Sound—Muskoka,11241.82,131635.50,0.00,26270.55,2020,2
2,"Albas,Dan",Central Okanagan—Similkameen—Nicola,23496.62,157063.09,218.14,27590.29,2020,2
3,"Alghabra,Hon.Omar",Mississauga Centre,15712.72,122873.63,260.50,22883.34,2020,2
4,"Alleslev,Leona",Aurora—Oak Ridges—Richmond Hill,21415.73,108721.70,340.40,18800.21,2020,2
...,...,...,...,...,...,...,...,...
345,"Zann,Lenore",Cumberland—Colchester,257.00,119696.55,0.00,28185.45,2020,2
346,"Zimmer,Bob",Prince George—Peace River—Northern Rockies,45027.99,152694.50,381.34,33319.55,2020,2
347,"Zuberi,Sameer",Pierrefonds—Dollard,4005.63,104330.24,943.69,25528.53,2020,2
348,"d'Entremont,Chris",West Nova,17232.05,102937.77,126.86,24057.97,2020,2


In [684]:

df.to_json('../src/expenditure_data.json', orient='records', force_ascii=False)

In [685]:
df[[
    'year', 
    'quarter', 
    'name',
    'constituency_name',
    'constituency_size',
    'number_of_electors',
    '_employees_salaries',
    '_service_contracts',
    'total',
    '_travel',
    'member',
    'designated_traveller',
    '_hospitality',
    'distance',
    'party'
    ]].to_json('../src/expenditure_data.json', orient='records', force_ascii=False)

In [686]:
df.loc[df['_employees_salaries'].isna()]

,constituency_name,name,year,quarter,_employees_salaries,_service_contracts,_travel,_hospitality,_gifts,_advertising,...,repairs_and_maintenance,telecommunication_services,ten_percenters,total,training,minister,parliament,party,distance,district_year


In [687]:
spending_by_party_year = df.groupby(['year', 'party']).sum().reset_index()[['year', 'party', 'total', '_travel', '_hospitality', '_service_contracts', '_employees_salaries']].dropna()

In [688]:
spending_by_party_year.to_json('../src/spending_by_party_year.json', orient='records', force_ascii=False)

In [689]:
spending_by_party_year.head()

,year,party,total,_travel,_hospitality,_service_contracts,_employees_salaries
0,2012,Bloc Québécois,1394781.66,221732.75,32096.81,0.0,767786.92
1,2012,Canadian Alliance,2622179.89,665543.60,36173.94,0.0,1395062.28
2,2012,Conservative,39828509.48,8525763.63,530152.18,0.0,21188481.28
3,2012,Green Party,407403.26,63155.44,3112.76,0.0,267124.36
4,2012,Independent,1076199.75,129019.85,26709.16,0.0,632176.23


In [690]:
df.loc[(df['name'].str.contains('May,Elizabeth'))]

,constituency_name,name,year,quarter,_employees_salaries,_service_contracts,_travel,_hospitality,_gifts,_advertising,...,repairs_and_maintenance,telecommunication_services,ten_percenters,total,training,minister,parliament,party,distance,district_year
66792,Saanich—Gulf Islands,"May,Elizabeth",2012,4,267124.36,0.00,63155.44,3112.76,0.00,9740.72,...,1681.90,11977.49,2625.90,407403.26,118.10,False,41.0,Green Party,3576.681315,2019
66808,Saanich—Gulf Islands,"May,Elizabeth",2013,4,269964.42,0.00,62672.87,3046.57,0.00,7244.43,...,876.17,10954.61,4614.72,408521.42,0.00,False,41.0,Green Party,3576.681315,2019
66824,Saanich—Gulf Islands,"May,Elizabeth",2014,1,50203.89,132.50,15071.06,609.04,0.00,0.00,...,0.00,1159.14,0.00,82453.45,0.00,False,41.0,Green Party,3576.681315,2019
66840,Saanich—Gulf Islands,"May,Elizabeth",2014,2,125052.64,328.47,27984.83,780.83,0.00,3460.48,...,497.42,2320.49,898.59,184758.22,0.00,False,41.0,Green Party,3576.681315,2019
66856,Saanich—Gulf Islands,"May,Elizabeth",2014,3,196675.15,620.97,42415.96,1412.08,0.00,5383.77,...,497.42,7057.38,1813.61,291817.93,0.00,False,41.0,Green Party,3576.681315,2019
66872,Saanich—Gulf Islands,"May,Elizabeth",2014,4,266553.20,1109.44,62573.73,1708.31,159.80,9027.06,...,497.42,11380.46,1813.61,403049.74,0.00,False,41.0,Green Party,3576.681315,2019
66888,Saanich—Gulf Islands,"May,Elizabeth",2015,1,54733.33,2407.47,17724.81,19.97,0.00,940.84,...,0.00,1109.36,875.31,85480.73,0.00,False,41.0,Green Party,3576.681315,2019
66904,Saanich—Gulf Islands,"May,Elizabeth",2015,2,97663.88,10535.44,24915.31,19.97,0.00,4736.44,...,0.00,3974.09,875.31,158966.94,0.00,False,41.0,Green Party,3576.681315,2019
66920,Saanich—Gulf Islands,"May,Elizabeth",2015,3,154455.29,10730.44,28604.16,19.97,0.00,4736.44,...,0.00,6320.04,875.31,229853.97,337.50,False,41.0,Green Party,3576.681315,2019
66936,Saanich—Gulf Islands,"May,Elizabeth",2015,4,239452.61,11582.78,47314.92,55.67,80.00,8519.02,...,0.00,10915.14,1667.76,364299.34,1125.00,False,41.0,Green Party,3576.681315,2019


In [691]:
df.loc[df['constituency_name'] == 'York Centre'][['number_of_electors', 'year']]

,number_of_electors,year
87943,71753.0,2012
87946,71753.0,2013
87949,71753.0,2014
87952,71753.0,2014
87955,71753.0,2014
87958,71753.0,2014
87961,71753.0,2015
87964,71753.0,2015
87967,71753.0,2015
87970,71753.0,2015


In [692]:
df.loc[(df['quarter'] == 4) & (df['year'] == 2020)][['name', 'constituency_name']].sort_values(by='name')['constituency_name']

23539                              Edmonton Manning
58022                           Parry Sound—Muskoka
14861           Central Okanagan—Similkameen—Nicola
46923                            Mississauga Centre
2783                Aurora—Oak Ridges—Richmond Hill
                            ...                    
18928                         Cumberland—Colchester
61427    Prince George—Peace River—Northern Rockies
59530                           Pierrefonds—Dollard
84695                                     West Nova
46665                                        Milton
Name: constituency_name, Length: 359, dtype: object